In [171]:
# -*- coding: utf-8 -*-
from pandas import read_csv
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing  # 预处理模块
import numpy as np
import pandas as pd
import time

import warnings
warnings.filterwarnings("ignore")

# 加载数据集与预处理

In [381]:
#加载训练集
df_base=read_csv("data/base-train.csv",encoding='gbk')# 加载base数据集，gbk编码读取中文字符
df_knowledge=read_csv("data/knowledge-train.csv")#加载knowlege数据集
df_money=read_csv("data/money-train.csv")#加载money数据集
df_year=read_csv("data/year-train.csv",encoding='gbk')#加载year数据集

#同样方法加载测试集
df_baseT=read_csv("data/base-test.csv")# 加载base数据集，gbk编码读取中文字符
df_knowledgeT=read_csv("data/knowledge-test.csv")#加载knowlege数据集
df_moneyT=read_csv("data/money-test.csv")#加载money数据集
df_yearT=read_csv("data/year-test.csv",encoding='gbk')#加载year数据集

In [282]:
df_base

,ID,注册时间,注册资本,行业,区域,企业类型,控制人类型,控制人持股比例,flag
0,5986361,2014.0,7090.0,服务业,湖北,有限责任公司,自然人,0.93,0
1,5991749,2007.0,5940.0,零售业,湖南,合伙企业,企业法人,0.57,0
2,5998154,2002.0,9720.0,工业,福建,合伙企业,自然人,0.74,0
3,5984390,2000.0,4800.0,商业服务业,山东,股份有限公司,NaN,0.90,0
4,5980535,2004.0,4530.0,零售业,广东,农民专业合作社,自然人,0.95,0
...,...,...,...,...,...,...,...,...,...
28514,2253874,2012.0,110.0,工业,广东,股份有限公司,自然人,0.89,1
28515,2823789,2000.0,3110.0,服务业,湖南,股份有限公司,自然人,0.64,1
28516,1680634,2003.0,1030.0,服务业,山东,农民专业合作社,企业法人,0.55,1
28517,101731,2004.0,5260.0,零售业,湖北,集体所有制企业,自然人,0.62,1


#### 填补缺失值

In [342]:
#前向填充，默认参数按行来，即使用上一行的值
df_base=df_base.fillna(method='ffill')
df_knowledge=df_knowledge.fillna(method='ffill')
df_money=df_money.fillna(method='ffill')
df_year=df_year.fillna(method='ffill')
df_baseT=df_baseT.fillna(method='ffill')
df_knowledgeT=df_knowledgeT.fillna(method='ffill')
df_moneyT=df_moneyT.fillna(method='ffill')
df_yearT=df_yearT.fillna(method='ffill')

In [343]:
df_baseT.shape

(7132, 8)

#### 按年份拆分money和year两个数据集  
```
(['ID', 'year', '债权融资额度', '债权融资成本', '股权融资额度', '股权融资成本', '内部融资和贸易融资额度','内部融资和贸易融资成本', '项目融资和政策融资额度', '项目融资和政策融资成本'],dtype='object')
(['ID', 'year', '从业人数', '资产总额', '负债总额', '营业总收入', '主营业务收入', '利润总额', '纳税总额','所有者权益合计'],dtype='object')
```
money数据集拆分为money15,money16,money16三个不同的数据集  
year数据集拆分为year15,year16,year16三个不同的数据集

In [382]:
#取year这一列中值为2015的样本
df_money15=df_money[df_money['year'].isin([2015])]
#删除year这一列
df_money15=df_money15.drop(columns=['year'])
#除ID外，修改其余列名
df_money15.columns=['ID','债权融资额度2015', '债权融资成本2015', '股权融资额度2015', '股权融资成本2015', '内部融资和贸易融资额度2015','内部融资和贸易融资成本2015', '项目融资和政策融资额度2015', '项目融资和政策融资成本2015']

df_money16=df_money[df_money['year'].isin([2016])]
df_money16=df_money16.drop(columns=['year'])
df_money16.columns=['ID', '债权融资额度2016', '债权融资成本2016', '股权融资额度2016', '股权融资成本2016', '内部融资和贸易融资额度2016','内部融资和贸易融资成本2016', '项目融资和政策融资额度2016', '项目融资和政策融资成本2016']

df_money17=df_money[df_money['year'].isin([2017])]
df_money17=df_money17.drop(columns=['year'])
df_money17.columns=['ID',  '债权融资额度2017', '债权融资成本2017', '股权融资额度2017', '股权融资成本2017', '内部融资和贸易融资额度2017','内部融资和贸易融资成本2017', '项目融资和政策融资额度2017', '项目融资和政策融资成本2017']

df_year15=df_year[df_year['year'].isin([2015])]
df_year15=df_year15.drop(columns=['year'])
df_year15.columns=['ID', '从业人数2015', '资产总额2015', '负债总额2015', '营业总收入2015', '主营业务收入2015', '利润总额2015', '纳税总额2015','所有者权益合计2015']


df_year16=df_year[df_year['year'].isin([2016])]
df_year16=df_year16.drop(columns=['year'])
df_year16.columns=['ID', '从业人数2016', '资产总额2016', '负债总额2016', '营业总收入2016', '主营业务收入2016', '利润总额2016', '纳税总额2016','所有者权益合计2016']


df_year17=df_year[df_year['year'].isin([2017])]
df_year17=df_year17.drop(columns=['year'])
df_year17.columns=['ID', '从业人数2017', '资产总额2017', '负债总额2017', '营业总收入2017', '主营业务收入2017', '利润总额2017', '纳税总额2017','所有者权益合计2017']


#同样方法处理测试集

df_money15T=df_moneyT[df_moneyT['year'].isin([2015])]
df_money15T=df_money15T.drop(columns=['year'])
df_money15T.columns=['ID','债权融资额度2015', '债权融资成本2015', '股权融资额度2015', '股权融资成本2015', '内部融资和贸易融资额度2015','内部融资和贸易融资成本2015', '项目融资和政策融资额度2015', '项目融资和政策融资成本2015']

df_money16T=df_moneyT[df_moneyT['year'].isin([2016])]
df_money16T=df_money16T.drop(columns=['year'])
df_money16T.columns=['ID',  '债权融资额度2016', '债权融资成本2016', '股权融资额度2016', '股权融资成本2016', '内部融资和贸易融资额度2016','内部融资和贸易融资成本2016', '项目融资和政策融资额度2016', '项目融资和政策融资成本2016']


df_money17T=df_moneyT[df_moneyT['year'].isin([2017])]
df_money17T=df_money17T.drop(columns=['year'])
df_money17T.columns=['ID',  '债权融资额度2017', '债权融资成本2017', '股权融资额度2017', '股权融资成本2017', '内部融资和贸易融资额度2017','内部融资和贸易融资成本2017', '项目融资和政策融资额度2017', '项目融资和政策融资成本2017']


df_year15T=df_yearT[df_yearT['year'].isin([2015])]
df_year15T=df_year15T.drop(columns=['year'])
df_year15T.columns=['ID',  '从业人数2015', '资产总额2015', '负债总额2015', '营业总收入2015', '主营业务收入2015', '利润总额2015', '纳税总额2015','所有者权益合计2015']


df_year16T=df_yearT[df_yearT['year'].isin([2016])]
df_year16T=df_year16T.drop(columns=['year'])
df_year16T.columns=['ID', '从业人数2016', '资产总额2016', '负债总额2016', '营业总收入2016', '主营业务收入2016', '利润总额2016', '纳税总额2016','所有者权益合计2016']


df_year17T=df_yearT[df_yearT['year'].isin([2017])]
df_year17T=df_year17T.drop(columns=['year'])
df_year17T.columns=['ID', '从业人数2017', '资产总额2017', '负债总额2017', '营业总收入2017', '主营业务收入2017', '利润总额2017', '纳税总额2017','所有者权益合计2017']


#### 按ID合并数据集

In [383]:
df1=pd.merge(df_base,df_knowledge,on='ID',how='outer')#on表示按那个属性合并，how=outer表示保留两个df的所有信息
df2=pd.merge(df1,df_money15,on='ID',how='outer')
df3=pd.merge(df2,df_money16,on='ID',how='outer')
df4=pd.merge(df3,df_money17,on='ID',how='outer')
df5=pd.merge(df4,df_year15,on='ID',how='outer')
df6=pd.merge(df5,df_year16,on='ID',how='outer')
df=pd.merge(df6,df_year17,on='ID',how='outer')

#同样方式合并测试集
df1T=pd.merge(df_baseT,df_knowledgeT,on='ID',how='outer')
df2T=pd.merge(df1T,df_money15T,on='ID',how='outer')
df3T=pd.merge(df2T,df_money16T,on='ID',how='outer')
df4T=pd.merge(df3T,df_money17T,on='ID',how='outer')
df5T=pd.merge(df4T,df_year15T,on='ID',how='outer')
df6T=pd.merge(df5T,df_year16T,on='ID',how='outer')
dfT=pd.merge(df6T,df_year17T,on='ID',how='outer')

In [384]:
dfT.shape

(7132, 59)

#### 将flag列放到最后一列

In [385]:
cols=list(df.columns)
#将flag列删除并添加到cols最后一列
cols.insert(len(cols)-1,cols.pop(cols.index('flag')))

#改变列顺序为cols
df=df.loc[:,cols] #获取所有的行，按照cols列表来依次取原df的列，得到新的DataFrame赋给新的变量df
# df.columns

Index(['ID', '注册时间', '注册资本', '行业', '区域', '企业类型', '控制人类型', '控制人持股比例', '专利',
       '商标', '著作权', '债权融资额度2015', '债权融资成本2015', '股权融资额度2015', '股权融资成本2015',
       '内部融资和贸易融资额度2015', '内部融资和贸易融资成本2015', '项目融资和政策融资额度2015',
       '项目融资和政策融资成本2015', '债权融资额度2016', '债权融资成本2016', '股权融资额度2016',
       '股权融资成本2016', '内部融资和贸易融资额度2016', '内部融资和贸易融资成本2016', '项目融资和政策融资额度2016',
       '项目融资和政策融资成本2016', '债权融资额度2017', '债权融资成本2017', '股权融资额度2017',
       '股权融资成本2017', '内部融资和贸易融资额度2017', '内部融资和贸易融资成本2017', '项目融资和政策融资额度2017',
       '项目融资和政策融资成本2017', '从业人数2015', '资产总额2015', '负债总额2015', '营业总收入2015',
       '主营业务收入2015', '利润总额2015', '纳税总额2015', '所有者权益合计2015', '从业人数2016',
       '资产总额2016', '负债总额2016', '营业总收入2016', '主营业务收入2016', '利润总额2016',
       '纳税总额2016', '所有者权益合计2016', '从业人数2017', '资产总额2017', '负债总额2017',
       '营业总收入2017', '主营业务收入2017', '利润总额2017', '纳税总额2017', '所有者权益合计2017',
       'flag'],
      dtype='object')

#### 非数值型特征获取

```
df.columns=["ID","注册时间","注册资本","行业","区域","企业类型","控制人类型","控制人持股比例","flag"]  
industry={nan, '交通运输业', '商业服务业', '工业', '服务业', '社区服务', '零售业'}
location={nan, '山东', '广东', '广西', '江西', '湖北', '湖南', '福建'}
companyType={nan, '农民专业合作社', '合伙企业', '有限责任公司', '股份有限公司', '集体所有制企业'}
controllerType={nan, '企业法人', '自然人'}
```

#### 映射有序特征

In [348]:
industry_mapping={
    '交通运输业':1,
    '商业服务业':2,
    '工业':3,
    '服务业':4,
    '社区服务':5,
    '零售业':6
}
location_mapping={
    '山东':1,
    '广东':2,
    '广西':3,
    '江西':4,
    '湖北':5,
    '湖南':6,
    '福建':7
}
companyType_mapping={
    '农民专业合作社':1,
    '合伙企业':2,
    '有限责任公司':3,
    '股份有限公司':4,
    '集体所有制企业':5
}
controllerType_mapping={
    '企业法人':1,
    '自然人':2,
}

In [386]:
df['行业']=df['行业'].map(industry_mapping)
df['区域']=df['区域'].map(location_mapping)
df['企业类型']=df['企业类型'].map(companyType_mapping)
df['控制人类型']=df['控制人类型'].map(controllerType_mapping)
df=df.fillna(method='ffill')

#同样方式处理测试集
dfT['行业']=dfT['行业'].map(industry_mapping)
dfT['区域']=dfT['区域'].map(location_mapping)
dfT['企业类型']=dfT['企业类型'].map(companyType_mapping)
dfT['控制人类型']=dfT['控制人类型'].map(controllerType_mapping)
dfT=dfT.fillna(method='ffill')

In [351]:
df.isnull().any()

ID                 False
注册时间               False
注册资本               False
行业                 False
区域                 False
企业类型               False
控制人类型              False
控制人持股比例            False
专利                 False
商标                 False
著作权                False
债权融资额度2015         False
债权融资成本2015         False
股权融资额度2015         False
股权融资成本2015         False
内部融资和贸易融资额度2015    False
内部融资和贸易融资成本2015    False
项目融资和政策融资额度2015    False
项目融资和政策融资成本2015    False
债权融资额度2016         False
债权融资成本2016         False
股权融资额度2016         False
股权融资成本2016         False
内部融资和贸易融资额度2016    False
内部融资和贸易融资成本2016    False
项目融资和政策融资额度2016    False
项目融资和政策融资成本2016    False
债权融资额度2017         False
债权融资成本2017         False
股权融资额度2017         False
股权融资成本2017         False
内部融资和贸易融资额度2017    False
内部融资和贸易融资成本2017    False
项目融资和政策融资额度2017    False
项目融资和政策融资成本2017    False
从业人数2015           False
资产总额2015           False
负债总额2015           False
营业总收入2015          False
主营业务收入2015         False


#### 获得训练向量$X(n\_samples, n\_features)$和目标值$y(n\_samples)$

In [387]:
values=df.values

X=values[:,1:-1]
y=values[:,-1]

#无标签测试向量
valuesT=dfT.values
X_t=valuesT[:,1:]
print(X_t)

#按8:2分割数据集为训练集和测试集，固定随机种子
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=2020)

#对数据集进行标准化
scaler=preprocessing.StandardScaler().fit(X_train)#通过训练集获得归一化函数
#使用相同的归一化函数处理训练向量和测试向量，以及无标签测试向量
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)
X_t=scaler.transform(X_t)
# X_t

#训练集中正例和负例个数
# y_list=y_train.tolist()
# print("正例：",y_list.count(0))
# print("负例：",y_list.count(1))

[[ 2.010000e+03  8.790000e+03  3.000000e+00 ...  3.414915e+05
   0.000000e+00 -1.406400e+05]
 [ 2.005000e+03  2.700000e+02  4.000000e+00 ...  3.110400e+03
   1.866240e+04 -2.835000e+03]
 [ 2.012000e+03  2.300000e+02  4.000000e+00 ...  2.070000e+02
   4.140000e+02 -1.840000e+03]
 ...
 [ 2.010000e+03  1.280000e+03  2.000000e+00 ...  8.960000e+03
   0.000000e+00 -1.536000e+04]
 [ 2.000000e+03  8.320000e+03  2.000000e+00 ...  1.810432e+05
   0.000000e+00  8.320000e+03]
 [ 2.004000e+03  2.300000e+03  1.000000e+00 ...  2.242500e+04
   0.000000e+00 -4.140000e+04]]


# 训练模型  
使用SVM作为模型时，通常采用如下流程：  

- 对样本数据进行归一化
- 应用核函数对样本进行映射
- 用cross-validation和grid-search对超参数进行优选
- 用最优参数训练得到模型
- 测试

In [164]:
#SVM rbf核函数，C，gamma参数调优
grid = GridSearchCV(SVC(), param_grid={"C":[0.1, 1, 10], "gamma":[1, 0.1, 0.01]}, cv=4)  
grid.fit(X_train, y_train)  
print("The best parameters are %s with a score of %0.2f" %(grid.best_params_, grid.best_score_))

The best parameters are {'C': 10, 'gamma': 0.01} with a score of 0.96


The best parameters are {'C': 10, 'gamma': 0.01} with a score of 0.96

In [388]:
#SVM模型训练
clf=SVC(C=10,gamma=0.01)
t1=time.time()
clf.fit(X_train,y_train)
t2=time.time()
print("训练时间： %fs"%(t2-t1))
sv=clf.n_support_
print("支持向量：",sv)

#支持向量中正例负例个数
print("正例：",sv[0])
print("负例：",sv[1])

训练时间： 16.388002s
支持向量： [1961 2786]
正例： 1961
负例： 2786


In [389]:
accur=clf.score(X_test,y_test)
print("准确率:",accur)

准确率: 0.9680925666199158


# 预测无标签数据集

In [390]:
#预测标签向量
y_pre=clf.predict(X_t)

In [396]:
# pd.DataFrame(valuesT)
list(y_pre).count(1)

2993

#### 添加id列

In [397]:
ID=valuesT[:,0]
df1=pd.DataFrame(ID)
df1.columns=['ID']
df1

,ID
0,5991927.0
1,5998351.0
2,5992703.0
3,5979231.0
4,5995422.0
...,...
7127,80564.0
7128,978515.0
7129,568065.0
7130,1889883.0


#### 合并id列与flag列

In [404]:
df2=pd.DataFrame(y_pre)
df2.columns=['flag']
df=pd.merge(df1,df2,left_index=True,right_index=True)#按行索引合并

,ID,flag
0,5991927.0,0.0
1,5998351.0,0.0
2,5992703.0,1.0
3,5979231.0,0.0
4,5995422.0,1.0
...,...,...
7127,80564.0,1.0
7128,978515.0,1.0
7129,568065.0,1.0
7130,1889883.0,1.0


In [409]:
df=df.astype(int)

In [410]:
df

,ID,flag
0,5991927,0
1,5998351,0
2,5992703,1
3,5979231,0
4,5995422,1
...,...,...
7127,80564,1
7128,978515,1
7129,568065,1
7130,1889883,1


#### 输出结果写入到文件

In [411]:
df.to_csv("output/res.txt",index=False)#输出到文件，不加行索引